# Criando Grafos com Apache Graphx

### Criação de Grafos com a bilbioteca GraphX do Spark e GraphFrames
![Alt Text](./img/graphx-logo.svg) 

In [1]:
# Carrega as bibliotecas de GraphFrames e loggin do scala usada pelo GrapgFrames
import os
jars    = ["jars/graphframes-0.5.0-spark2.1-s_2.11.jar", "jars/scala-logging-api_2.11-2.1.2.jar", "jars/scala-logging-slf4j_2.11-2.1.2.jar"]
pyfiles = ["pyfiles/graphframes.zip"]
os.environ["PYSPARK_SUBMIT_ARGS"] = " --jars "+ ",".join(jars)  +" --py-files "+ ",".join(pyfiles)  +"  pyspark-shell"

In [2]:
import sys
import findspark
import json, random

import pandas as pd
import numpy as np
from IPython.core.display import HTML
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.types import StructType,StructField,ArrayType

In [3]:
spark = SparkSession\
    .builder\
    .appName("SparkGraphx")\
    .getOrCreate()
spark.sparkContext.setCheckpointDir("log")
spark.sparkContext

### Leitura dos dados de usuários (User.csv) e conexão entre os usuários (UserGraph.csv)

In [4]:
user_df = spark.read.format("csv").option("header", "true")\
                .load("dataset/User.csv")\
                .withColumnRenamed("ID", "id")
        
user_df.show(2)

+----+------+
|  id|  NAME|
+----+------+
|1090|Jessie|
|1159|Melvin|
+----+------+
only showing top 2 rows



In [5]:
user_graph_df = spark.read.format("csv").option("header", "true")\
                .load("dataset/UserGraph.csv")\
                .withColumnRenamed("USER_1", "src")\
                .withColumnRenamed("USER_2", "dst")
            
user_graph_df.show(2)

+----+----+
| src| dst|
+----+----+
|1090|5309|
|1090|3547|
+----+----+
only showing top 2 rows



### Criando Grafo a partir dos dados - Graph 

In [6]:
from graphframes import *

In [7]:
# Carrega os dados para a Classe do GraphFrame
#
g = GraphFrame(user_df, user_graph_df)
print("Total de Vertices (usuários): %d" % g.vertices.count())
print("Total de Arestas (Conexões): %d" % g.edges.count())
g.cache()

Total de Vertices (usuários): 6486
Total de Arestas (Conexões): 336534


GraphFrame(v:[id: string, NAME: string], e:[src: string, dst: string])

Vamos descobrir **qual é o usuário com maior quantidade de conexões**, ou seja, qual pessoa tem mais amigos nessa rede social. Para isso utilizamos a $inDegress$, que retorna a quantidade de conexões entre os vértices.

In [8]:
def get_most_connected(g, topN):
    '''
    Return a list containing the most friends
    '''
    g_indegrees = g.inDegrees
    return g.vertices.join(g_indegrees, "id").orderBy("inDegree", ascending=False).limit(topN)

In [9]:
most_connected = get_most_connected(g, 1000)
most_connected.show(10)

+----+---------+--------+
|  id|     NAME|inDegree|
+----+---------+--------+
| 859|   Hallie|    1933|
|5306|     Arch|    1741|
|2664|     Edna|    1528|
|5716|   Dalton|    1426|
|6306| Napoleon|    1394|
|3805|Arlington|    1386|
|2557|    Giles|    1371|
|4898| Gottlieb|    1345|
|5736|    Lemon|    1289|
| 403|    Alvah|    1280|
+----+---------+--------+
only showing top 10 rows



Ou seja, o usuário "Hallie" tem um total de 1933 conexões.  Para exibir os usuários conectados a "Hallie", podemos usar a função $find$, que utiliza alguns modificadores para mapear a busca em Gráfos. Ex:
$$
(a)-[e]->(b)
$$
significa que, um vertice qualquer "a", deve estar ligado por uma aresta "e" a um vértice qualquer "b". Dessa forma é possível construir diveras Query diferentes.

In [10]:
def get_users_connected(g, user_id):
    '''
    Return a list of connected users
    '''
    return g.find("(a)-[e]->(b)").filter("b.id = %d" % user_id).select("a.id", "a.NAME")

users = get_users_connected(g, 859)
print("Total de usuários conectados a 'Hallie': ", users.count())
users.show(5)

Total de usuários conectados a 'Hallie':  1933
+----+-------+
|  id|   NAME|
+----+-------+
|1572| Finley|
|2069|Patrick|
|2904|   Linn|
|3210|Preston|
|3606|Colonel|
+----+-------+
only showing top 5 rows



Podemos usar uma consulta semelhante pra encontrar amigos em comum entre dois usuários que não estão conecatados. O famoso "pessoas que talvez você conheça". Algo como:
$$
(a)-[e]->(b); (b)-[e2]->(c); !(a)-[]->(c)
$$
Ou seja, queremos um usuário "a" que tenha conexão com "b" [(a)-[e]->(b);], sendo que "b" tem conexão com "c", mas [(b)-[e2]->(c);], mas "a" e "c" não tenha conexão [!(c)-[]->(a)].

In [11]:
def get_friends_suggestion(g, user_id):
    '''
    Returns a list of suggested friendships "people you may know"
    '''
    users = g.find("(a)-[e]->(b); (b)-[e2]->(c); !(a)-[]->(c)").filter("a.id = %d" % user_id)
    return users.select("c.id", "c.name")

users = get_friends_suggestion(g, 859).cache()
print("Total de possíveis amigos: ", users.count())
users.show(5)

Total de possíveis amigos:  56331
+----+--------+
|  id|    name|
+----+--------+
|4845|  Winnie|
|3219|    Juan|
|1310|Laurence|
| 177|Nicholas|
|1003|  George|
+----+--------+
only showing top 5 rows



#### Subgraphs

Os sub-grafos são uma amostra do grafo original, um subgrupo que pode ter interpretação própria. Vamos separa um sub-grafo dos usuários que tem conexão entre dois outros usuários. Ou seja, dado dois usuários, gostaria de saber todos os usuários que tem conexão.

In [12]:
def get_suggraphs_between_users(g, user1_id, user2_id):
    '''
    Return sugraphs conections between user1 and user2
    '''
    g_user  = g.find("(a)-[e]->(b); (c)-[e2]->(b)")\
                .filter("a.id = {} and c.id = {}".format(user1_id, user2_id))

    g_vert  = g_user.select("a.id", "a.NAME")\
                .unionAll(g_user.select("b.id", "b.NAME"))\
                .unionAll(g_user.select("c.id", "c.NAME")).distinct()

    g_edges = g_user.select("e.src", "e.dst")\
                .unionAll(g_user.select("e2.src", "e2.dst")).distinct()

    g2 = GraphFrame(g_vert, g_edges)

    return g2

In [13]:
user1_id = 4845 # Winnie
user2_id = 1572 # Finley

# SubGraph with Winnie connections between Finley
g_users = get_suggraphs_between_users(g, user1_id, user2_id)

In [14]:
g_users.vertices.show()

+----+------+
|  id|  NAME|
+----+------+
|  10|Edward|
| 337|Josiah|
|4845|Winnie|
|1572|Finley|
|2548|  Dora|
+----+------+



Lista com os usuáriso que conexão com $user1_id$ e $user2_id$, e o Sub-Graph dessa dessa conexão.

![Alt Text](./img/graph.png) 

Essa análise pode ser útil para identificar possíveis amigos em comum entre dois usuários.

#### Salvar GraphFrames em Json para visualização da biblioteca 3d


In [15]:
def graph2json(graph):
    '''
    Transform GraphFrames to Json File
    '''
    vertices = [{"id": v.id, "name": v.NAME, "group": 1} for v in graph.vertices.collect()]
    edges    = [{"source": v.src, "target": v.dst, "value": 5} for v in graph.edges.collect()]

    graph_json = {"nodes":vertices, "links": edges}
    return graph_json

def graph_to_file(graph, path):
    '''
    Save GraphFrames in Json File
    '''
    graph_json = graph2json(graph)
    with open(path, 'w') as file:
        json.dump(graph_json, file, ensure_ascii=False)
    return path

In [16]:
graph_to_file(g_users, "graph.json")

'graph.json'